Basic definitions

In [ ]:
using Plots

function Vol3SL(a0::Float64,a1::Float64,b::Float64)

    res = (a0^2 + a0*a1 + a1^2) * sqrt(complex((a0-a1)^2/2 - b^2))/3

    res

end

function Vol3TL(a0::Float64,a1::Float64,b::Float64)

    res = (a0^2 + a0*a1 + a1^2) * sqrt(complex((a0-a1)^2/2 + b^2))/3

    res

end

function generate_null(v::Array{Float64, 1})

    res = [cosh(asinh(-v[1])), cos(atan(v[3],v[2])) + v[1]*sin(atan(v[3],v[2])), -sin(atan(v[3],v[2]))+v[1]*cos(atan(v[3],v[2]))]

    res
    
end

function S_Regge_I(a0::Float64,a1::Float64,b::Float64)

    res = -4*abs(a0-a1)*log(abs(a0-a1) + sqrt(complex2(2*(a0-a1)^2 - 4*b^2))) 
    + 4*abs(b)*log((a0-a1)^2 + sqrt(complex(8*b^2*((a0-a1)^2 - 2*b^2)))) 
    + (4*abs(b) - 2*abs(a0 - a1))*log(4*b^2 - (a0-a1)^2)

    res

end

function S_Regge_null(a0::Float64,a1::Float64,b::Float64)

    res = 4*abs(a0 - a1)*log(abs(a0 - a1)) - 4*abs(b)*log((a0 - a1)^2/2)

    res

end

function S_Regge_II(a0::Float64,a1::Float64,b::Float64)

    res = -4*(a0 - a1)*asinh((a0 - a1)/sqrt(complex((a0 - a1)^2 - 4*b^2))) + 4*abs(b)*acosh((a0 - a1)^2/((a0 - a1)^2 - 4*b^2))

    res

end

function S_Regge_III(a0::Float64,a1::Float64,b::Float64)

    res = -4*(a0 - a1)*asinh((a0 - a1)/sqrt(complex((a0 - a1)^2 - 4*b^2))) + 4*abs(b)*(pi/2 - acos((a0 - a1)^2/(4*b^2 + (a0 - a1)^2)))

    res

end

function S_phi_SL(a0::Float64,a1::Float64,b::Float64,ϕ0::Float64,ϕ1::Float64,m::Float64)

    res = (a0 + a1)^2/(8*sqrt(complex((a0 - a1)^2/2 - b^2)))*(ϕ0 - ϕ1)^2 - m^2/2*Vol3SL(a0,a1,b)*(ϕ0^2 + ϕ1^2)/2

    res

end

function S_phi_TL(a0::Float64,a1::Float64,b::Float64,ϕ0::Float64,ϕ1::Float64,m::Float64)

    res = (a0 + a1)^2/(8*sqrt(complex((a0 - a1)^2/2 - b^2)))*(ϕ0 - ϕ1)^2 + m^2/2*Vol3TL(a0,a1,b)*(ϕ0^2 + ϕ1^2)/2

    res

end

function ϑa(a0::Float64,a1::Float64,b::Float64)

    res = sqrt(complex((a0 - a1 + sqrt(complex(2(a0 - a1)^2 - 4*b^2)))/(a0 - a1 - sqrt(complex(2(a0 - a1)^2 - 4*b^2)))))

    res

end

function ϑb(a0::Float64,a1::Float64,b::Float64)

    res = sqrt(complex(((a0 - a1)^2 - 2*abs(b)*sqrt(complex(2(a0 - a1)^2 - 4*b^2)))/((a0 - a1)^2 + 2*abs(b)*sqrt(complex(2(a0 - a1)^2 - 4*b^2)))))

    res

end

n13 = [0, 0, 1]
n14 = [0, -1, 0]
n15 = [0, 0, -1]
n16 = [0, 1, 0]
n23 = [0, 0, -1]
n24 = [0, 1, 0]
n25 = [0, 0, 1]
n26 = [0, -1, 0]

function n13_refl(a0::Float64, a1::Float64, b::Float64)

    res = [-(2*(a0 - a1)*sqrt(complex(2(a0 - a1)^2 - 4*b^2)))/((a0 - a1)^2 - 4*b^2), 0, (-3*(a0 - a1)^2 + 4*b^2)/((a0 - a1)^2 - 4*b^2)]

    res
    
end

function n15_refl(a0::Float64, a1::Float64, b::Float64)

    res = [(2*(a0 - a1)*sqrt(complex(2(a0 - a1)^2 - 4*b^2)))/((a0 - a1)^2 - 4*b^2), 0, (3*(a0 - a1)^2 - 4*b^2)/((a0 - a1)^2 - 4*b^2)]

    res
    
end

function n23_refl(a0::Float64, a1::Float64, b::Float64)

    res = [(2*(a0 - a1)*sqrt(complex(2(a0 - a1)^2 - 4*b^2)))/((a0 - a1)^2 - 4*b^2), 0, (3*(a0 - a1)^2 - 4*b^2)/((a0 - a1)^2 - 4*b^2)]

    res
    
end

function n25_refl(a0::Float64, a1::Float64, b::Float64)

    res = [-(2*(a0 - a1)*sqrt(complex(2(a0 - a1)^2 - 4*b^2)))/((a0 - a1)^2 - 4*b^2), 0, (-3*(a0 - a1)^2 + 4*b^2)/((a0 - a1)^2 - 4*b^2)]

    res
    
end

function n34_refl(a0::Float64, a1::Float64, b::Float64)

    res = [((5*(a0 - a1)^2 - 4*b^2)*sqrt(complex((a0 - a1)^2 -2*b^2)))/(sqrt(2)*(-(a0 - a1)^2*b + 4*b^3)), (a0 - a1)/(2*b), (7*(a0 - a1)^3 + 12*(-a0 + a1)*b^2)/(-2*(a0 - a1)^2*b + 8*b^3)]

    res
    
end

function n36_refl(a0::Float64, a1::Float64, b::Float64)

    res = [(sqrt(complex((a0 - a1)^2/2 - b^2)))/b, -(a0 - a1)/(2*b), (a0 - a1)/(2*b)]
    
end

function n45_refl(a0::Float64, a1::Float64, b::Float64)

    res = [((5*(a0 - a1)^2 - 4*b^2)*sqrt(complex((a0 - a1)^2 -2*b^2)))/(sqrt(2)*(-(a0 - a1)^2*b + 4*b^3)), -(a0 - a1)/(2*b), (7*(a0 - a1)^3 + 12*(-a0 + a1)*b^2)/(-2*(a0 - a1)^2*b + 8*b^3)]
    
    res

end

function n56_refl(a0::Float64, a1::Float64, b::Float64)

    res = [-(sqrt(complex((a0 - a1)^2/2 - b^2)))/b, -(a0 - a1)/(2*b), -(a0 - a1)/(2*b)]
    
end

function edgevectorsets_refl(a0::Float64, a1::Float64, b::Float64)

    res = [ [0, 0, n13_refl(a0, a1, b), n14, n15_refl(a0, a1, b), n16],
            [0, 0, n23_refl(a0, a1, b), n24, n25_refl(a0, a1, b), n26],
            [n13_refl(a0, a1, b), n23, 0, n34_refl(a0, a1, b), 0, n36_refl(a0, a1, b)],
            [n14, n24, n34_refl(a0, a1, b), 0, n45_refl(a0, a1, b), 0],
            [n15_refl(a0, a1, b), n25_refl(a0, a1, b), 0, n45_refl(a0, a1,b), 0, n56_refl(a0, a1, b)],
            [n16, n26, n36_refl(a0, a1, b), 0, n56_refl(a0, a1, b), 0]]

    res
    
end

function nullvectorsets_refl(a0::Float64, a1::Float64, b::Float64)

    res = [ [0, 0, generate_null(n13_refl(a0, a1, b)), generate_null(n14), generate_null(n15_refl(a0, a1, b)), generate_null(n16)],
            [0, 0, generate_null(n23_refl(a0, a1, b)), generate_null(n24), generate_null(n25_refl(a0, a1, b)), generate_null(n26)],
            [generate_null(n13_refl(a0, a1, b)), generate_null(n23), 0, generate_null(n34_refl(a0, a1, b)), 0, generate_null(n36_refl(a0, a1, b))],
            [generate_null(n14), generate_null(n24), generate_null(n34_refl(a0, a1, b)), 0, generate_null(n45_refl(a0, a1, b)), 0],
            [generate_null(n15_refl(a0, a1, b)), generate_null(n25_refl(a0, a1, b)), 0, generate_null(n45_refl(a0, a1,b)), 0, generate_null(n56_refl(a0, a1, b))],
            [generate_null(n16), generate_null(n26), generate_null(n36_refl(a0, a1, b)), 0, generate_null(n56_refl(a0, a1, b)), 0]]

    res
    
end

indexsets = [[3, 4, 5, 6], [3, 4, 5, 6], [1, 2, 4, 6], [1, 2, 3, 5], [1, 2, 4, 6], [1, 2, 3, 5]]

function lengthsetsSL(a0::Float64, a1::Float64, b::Float64)
    
    res = [[0, 0, a0, a0, a0, a0], [0, 0, a1, a1, a1, a1], [a0, a1, 0, b, 0, b], [a0, a1, b, 0, b, 0], [a0, a1, 0, b, 0, b], [a0, a1, b, 0, b, 0]]

    res

end

function ϑsets(a0::Float64, a1::Float64, b::Float64)

    res = [ [0, 0, ϑa(a0, a1, b)^(-2), ϑa(a0, a1, b)^(-2), ϑa(a0, a1, b)^(-2), ϑa(a0, a1, b)^(-2)],
            [0, 0, ϑa(a0, a1, b)^2, ϑa(a0, a1, b)^2, ϑa(a0, a1, b)^2, ϑa(a0, a1, b)^2],
            [ϑa(a0, a1, b)^(-2), ϑa(a0, a1, b)^2, 0, ϑb(a0, a1, b)^(-2), 0 ,ϑb(a0, a1, b)^(-2)],
            [ϑa(a0, a1, b)^(-2), ϑa(a0, a1, b)^2, ϑb(a0, a1, b)^(-2), 0 ,ϑb(a0, a1, b)^(-2), 0],
            [ϑa(a0, a1, b)^(-2), ϑa(a0, a1, b)^2, 0, ϑb(a0, a1, b)^(-2), 0 ,ϑb(a0, a1, b)^(-2)],
            [ϑa(a0, a1, b)^(-2), ϑa(a0, a1, b)^2, ϑb(a0, a1, b)^(-2), 0 ,ϑb(a0, a1, b)^(-2), 0]]

    res

end

function detTL(a0::Float64, a1::Float64, b::Float64)

    res = 1/b^4 * (1/256 + im/256) * a0^3 * (a0 - a1)^4 * a1^3 * ((a0 - a1)^2 + 4b^2)^3 * ((2 + im)*a1^2 - 4*im*b^2 + 2(1 + im)*a1*sqrt(complex(2*(a0 - a1)^2 + 4*b^2))
    +a0^2*(-2(1 + im)*a1 + (1 + im)*(2*b + sqrt(complex(2*(a0 - a1)^2 +4*b^2)))) + a1*((2 + im)*a1^2 + 4*im*b^2 + (1 + im)*a1*(2*b + sqrt(complex(2*(a0-a1)^2 + 4*b^2)))))
    
    res
    
end



    





: 